In [1]:
# STANDARD LIBRARIES
import pandas as pd
from pandas._testing import assert_series_equal
import numpy as np
import pickle
import datetime as dt

# GEOLOCATION
import geopandas as gpd
from geopandas.tools import geocode
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from pyzipcode import ZipCodeDatabase


# FEATURE ENGINEERING AND PREPROCESSING
from sklearn.preprocessing import OneHotEncoder

# Read Files

In [54]:
# FROM CLEANING NOTEBOOK

data = pd.read_csv("../data/clean-data/data-clean.csv", low_memory=False)
pw_df = pd.read_csv("../data/clean-data/pw_df-clean.csv", low_memory=False)
requests_df = pd.read_csv("../data/clean-data/requests_df-clean.csv")
fire_df = pd.read_csv("../data/clean-data/fire_df-clean.csv")

df_list = [
    data,
    pw_df,
    requests_df,
    fire_df
]

for df in df_list:
    try:
        df.drop(columns="Unnamed: 0", inplace=True)
    except KeyError:
        pass

In [4]:
# READ ENG FILES:

# data = pd.read_csv("../data/clean-data/data-engineered.csv")
pw_df = pd.read_csv("../data/clean-data/pw_df-engineered.csv")
fire_df = pd.read_csv("../data/clean-data/fire_df-engineered.csv")
requests_df = pd.read_csv("../data/clean-data/requests_df-engineered.csv")
# main = pd.read_csv("../data/clean-data/main-engineered.csv")

df_list = [
#     data,
    pw_df,
    requests_df,
    fire_df,
#     main
]


for df in df_list:
    try:
        df.drop(columns="Unnamed: 0", inplace=True)
    except KeyError:
        pass

# data_1 = pd.read_csv("../data/clean-data/data_1-eng.csv")

# Master Function, Address & Geocodes

## Edit Function

In [19]:
def create_address_list(df):
    
    init_check = input("Do you have only one column to create full addresses? Enter 'yes' or 'no': ")
    
    if init_check.lower() == "yes":
        status = True
        while status:
            try:
                col = input("Enter a column to create address list: ")
                address_list = list(df[col])
                return address_list
            except KeyError:
                print("Please enter a valid column to create list") 
                check_status = input("Would you like to continue? Enter 'yes' or 'no'")
                if check_status == "no":
                    status = False
                else:
                    status = True
    else:
        address_list = []
        return address_list

    
# SOME ADDRESSES NEED FURTHER CLEANING
def clean_addresses(df, col="full_address"):
        
    df[col] = df[col].replace({
        " st ": " street",
        " pk ": " park",
        " av ": " avenue",
        " ave ": " avenue",
        " ct ": " court",
        " dr ": " drive",
        " rd ": " road",
        " te ": " terrace",
        " cir ": " circle",
        " sq ": " square",
        " a ": " ",
        " b ": " ",
        " c ": " ",
        " d ": " ",
        " e ": " ",
        " f ": " ",
        " g ": " ",
        " h ": " ",
        " i ": " ",
        " j ": " ",
        " k ": " ",
        " j ": " ",
        " k ": " ",
        " l ": " ",
        " m ": " ",
        " n ": " ",
        " o ": " ",
        " p ": " ",
        " q ": " ",
        " r ": " ",
        " s ": " ",
        " t ": " ",
        " u ": " ",
        " v ": " ",
        " w ": " ",
        " x ": " ",
        " y ": " ",
        " z ": " "
    },
    regex=True)
    
    return df    
    


def create_address_geolocator(df):
    
    address_list = create_address_list(df)
    
    if len(address_list) > 0:
        for address in range(len(address_list)):
            address_list[address] += " boston, ma"
        df["full_address"] = address_list
    else:
        
        st_number = input("Enter street number column: ")
        st_name = input("Enter street name column: ")
        st_suffix = input("Enter street suffix column: ")
        
        print(st_number)
        df["full_address"] = df[st_number] + " " +\
                             df[st_name] + " " +\
                             df[st_suffix] + " " +\
                             "boston, ma"
    df.drop_duplicates(subset=["full_address"], inplace=True) #
    df.reset_index(inplace=True) #
    return df["full_address"]



def create_address_id(df):
    
    address_list = create_address_list(df)
    
    if len(address_list) > 0:
#         for address in range(len(address_list)):
#             address_list[address] += " boston, ma"
        df["full_address"] = address_list
        df["full_address"] = df["full_address"].str.split()
        df["full_address"] = df["full_address"].apply(lambda x: " ".join(x[:3])) 
        df["full_address"] = df["full_address"].astype(str) + " boston, ma"
        # https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column
    else:
        
        st_number = input("Enter street number column: ")
        st_name = input("Enter street name column: ")
        st_suffix = input("Enter street suffix column: ")
        
        print(st_number)
        df["full_address"] = df[st_number] + " " +\
                             df[st_name] + " " +\
                             df[st_suffix] + " " +\
                             "boston, ma"
    df.drop_duplicates(subset=["full_address"], inplace=True) #
    df.reset_index(inplace=True, drop=True) #
    return df



def combine_lat_long(df):
    try:
        df["location"] = list(zip(df["latitude"], df["longitude"]))
    except KeyError:
        df["location"] = list(zip(df["lat"], df["long"]))

        
def create_geocodes(df):
    
    geolocator = Nominatim(user_agent="my-application")
    geocode_list = []
    error_list = []
    counter = 0
    
    addresses = create_address_geolocator(df)
    addresses = pd.Series(addresses)
    df.drop_duplicates(subset=["full_address"], inplace=True) 
    
    init_check = input("Do you have existing lat/long coords"\
                       "for this dataframe? Enter yes or no: ")
    
    if init_check == "no":
        for address in addresses:
#             print(address)
            try:
                latitude = geolocator.geocode(address).latitude
                longitude= geolocator.geocode(address).longitude
                geocode_list.append((latitude, longitude))
                counter += 1
            except AttributeError:
                error_list.append(address)
                print(f"Could not create coordindates for {address}")
                pass
            except GeocoderTimedOut:
                if attempt <= max_attempts:
                    get_geocodes(address, attempt=attempt+1)
                raise
            if counter % 500 == 0:
                print(f"Completed Address Count: {counter}")
                
        df["location"] = pd.Series(geocode_list)
        
    else:
        df["location"] = combine_lat_long(df)


    return df, error_list
    # https://gis.stackexchange.com/questions/173569/avoid-time-out-error-nominatim-geopy-open-street-maps


## Run Functions

In [20]:
data = create_address_id(data)
# pw_df = create_address_id(pw_df)
# requests_df = create_address_id(requests_df)

KeyboardInterrupt: 

In [ ]:
data.zipcode

In [ ]:
data = clean_addresses(data, "full_address")
# pw_df = clean_addresses(pw_df, "full_address")
# requests_df = clean_addresses(requests_df, "full_address")

In [ ]:
# data = clean_addresses(data, "st_name_suf")
# pw_df = clean_addresses(pw_df, "st_name_suf")

# Feature Engineering

## Protected

In [ ]:
# COMPLETED:
    # IN ORDER TO EXPEDITE THIS, WILL DIVIDE THE DF UP INTO 10 CHUNKS
    # data_1
    
# data_1, data_2, data_3, data_4,data_5,\
# data_6, data_7, data_8, data_9, data_10,\
# data_11, data_12, data_13, data_14, data_15 = np.array_split(data, 15)

# data_2, error_list_2 = create_geocodes(data_2)

In [ ]:
# error_list_1 = error_list
# print(len(error_list_1))
# error_list_1

In [ ]:
# print(data_1.shape)
# data_1.head(20)

## Encode

In [58]:
data_encode_list = [
    "lu"
#     "zipcode"
]

pw_df_encode_list = [
#     "status",
    "description"
]

requests_encode_list = [
#     "reason",
    "source",
]

In [8]:
# BUG WITH THIS FUNCTION TAKING IN MORE THAN ONE COLUMN FROM LIST

def create_encoder(df, col_list):
    
#     init_check = input("Would you like to OneHotEncode this Dataset?"\
#                         " Enter 'yes' or 'no': ")
    df_encoder_list = []
    
#     if init_check == "yes":
    for col in col_list:
        df[col] = df[col].astype("category")
        df[col + "_category"] = df[col].cat.codes
        print(f"Created category column for column: {col}")

        df_encoder_list.append(col + "_category") 
        print(df_encoder_list)

        encoder = OneHotEncoder(
#             drop="first",
            handle_unknown="error"
        )
            
        enc_df = pd.DataFrame(encoder.fit_transform(df[df_encoder_list]).toarray())
        enc_df.columns = encoder.get_feature_names(df_encoder_list)
            
        df = pd.concat([df, enc_df], axis=1)
        # https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd
            
    return df
        

In [55]:
data = create_encoder(data, data_encode_list)

Created category column for column: lu
['lu_category']


In [56]:
pw_df = create_encoder(pw_df, pw_df_encode_list)

Created category column for column: description
['description_category']


In [59]:
requests_df = create_encoder(requests_df, requests_encode_list)

Created category column for column: source
['source_category']


## Binarize

In [60]:
data["own_occ"] = data["own_occ"].map({"n": 0, "y":1})

In [61]:
pw_df["status"] = pw_df["status"].map({"closed": 0, "open": 1})

In [62]:
requests_df["case_status"] = requests_df["case_status"].map({"closed": 0, "open": 1})

## Columns

### data

In [ ]:
# ==================================

In [63]:
data["zipcode"] = data["zipcode"].astype(str)
data["zipcode"] = data["zipcode"].apply(lambda x: "0" + x[:4])  

In [64]:
data["num_street"] = data[["st_num", "st_name"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
data["zip_street"] = data[["st_name", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
data["zip_num_street"] = data[["num_street", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
data["land_value"] = data.groupby("zip_num_street")["av_land"].transform('sum')
data["bldg_value"] = data.groupby("zip_num_street")["av_bldg"].transform('sum')
data["total_value"] = data.groupby("zip_num_street")["av_total"].transform('sum')
data["gross_tax_amount"] = data.groupby("zip_num_street")["gross_tax"].transform('sum')

In [65]:
lu_cat_dict = {
    0: "a",
    1: "ah",
    2: "c",
    3: "cc", 
    4: "cd", 
    5: "cl",
    6: "cm",
    7: "e",
    8: "ea",
    9: "i",
    10: "r1",
    11: "r2",
    12: "r3",
    13: "r4",
    14: "rc",
    15: "rl"
}

In [66]:
# GROUPING THE CATEGORICAL VARIABLES BY PROPERTY TOTALS
data["total_a_prop"] = data.groupby("zip_num_street")["lu_category_0"].transform('sum')
data["total_ah_prop"] = data.groupby("zip_num_street")["lu_category_1"].transform('sum')
data["total_c_prop"] = data.groupby("zip_num_street")["lu_category_2"].transform('sum')
data["total_cc_prop"] = data.groupby("zip_num_street")["lu_category_3"].transform('sum')
data["total_cd_prop"] = data.groupby("zip_num_street")["lu_category_4"].transform('sum')
data["total_cl_prop"] = data.groupby("zip_num_street")["lu_category_5"].transform('sum')
data["total_cm_prop"] = data.groupby("zip_num_street")["lu_category_6"].transform('sum')
data["total_e_prop"] = data.groupby("zip_num_street")["lu_category_7"].transform('sum')
data["total_ea_prop"] = data.groupby("zip_num_street")["lu_category_8"].transform('sum')
data["total_i_prop"] = data.groupby("zip_num_street")["lu_category_9"].transform('sum')
data["total_r1_prop"] = data.groupby("zip_num_street")["lu_category_10"].transform('sum')
data["total_r2_prop"] = data.groupby("zip_num_street")["lu_category_11"].transform('sum')
data["total_r3_prop"] = data.groupby("zip_num_street")["lu_category_12"].transform('sum')
data["total_r4_prop"] = data.groupby("zip_num_street")["lu_category_13"].transform('sum')
data["total_rc_prop"] = data.groupby("zip_num_street")["lu_category_14"].transform('sum')
data["total_rl_prop"] = data.groupby("zip_num_street")["lu_category_15"].transform('sum')

data["owner_occ_prop"] = data.groupby("zip_street")["own_occ"].transform('sum')

In [67]:
# GROUPING THE CATEGORICAL VARIABLES BY PROPERTY TOTALS
data["total_a_street"] = data.groupby("zip_street")["lu_category_0"].transform('sum')
data["total_ah_street"] = data.groupby("zip_street")["lu_category_1"].transform('sum')
data["total_c_street"] = data.groupby("zip_street")["lu_category_2"].transform('sum')
data["total_cc_street"] = data.groupby("zip_street")["lu_category_3"].transform('sum')
data["total_cd_street"] = data.groupby("zip_street")["lu_category_4"].transform('sum')
data["total_cl_street"] = data.groupby("zip_street")["lu_category_5"].transform('sum')
data["total_cm_street"] = data.groupby("zip_street")["lu_category_6"].transform('sum')
data["total_e_street"] = data.groupby("zip_street")["lu_category_7"].transform('sum')
data["total_ea_street"] = data.groupby("zip_street")["lu_category_8"].transform('sum')
data["total_i_street"] = data.groupby("zip_street")["lu_category_9"].transform('sum')
data["total_r1_street"] = data.groupby("zip_street")["lu_category_10"].transform('sum')
data["total_r2_street"] = data.groupby("zip_street")["lu_category_11"].transform('sum')
data["total_r3_street"] = data.groupby("zip_street")["lu_category_12"].transform('sum')
data["total_r4_street"] = data.groupby("zip_street")["lu_category_13"].transform('sum')
data["total_rc_street"] = data.groupby("zip_street")["lu_category_14"].transform('sum')
data["total_rl_street"] = data.groupby("zip_street")["lu_category_15"].transform('sum')

data["owner_occ_street"] = data.groupby("zip_street")["own_occ"].transform('sum')

In [68]:
# GROUPING THE CATEGORICAL VARIABLES BY ZIPCODE TOTALS
data["total_a_zipcode"] = data.groupby("zipcode")["lu_category_0"].transform('sum')
data["total_ah_zipcode"] = data.groupby("zipcode")["lu_category_1"].transform('sum')
data["total_c_zipcode"] = data.groupby("zipcode")["lu_category_2"].transform('sum')
data["total_cc_zipcode"] = data.groupby("zipcode")["lu_category_3"].transform('sum')
data["total_cd_zipcode"] = data.groupby("zipcode")["lu_category_4"].transform('sum')
data["total_cl_zipcode"] = data.groupby("zipcode")["lu_category_5"].transform('sum')
data["total_cm_zipcode"] = data.groupby("zipcode")["lu_category_6"].transform('sum')
data["total_e_zipcode"] = data.groupby("zipcode")["lu_category_7"].transform('sum')
data["total_ea_zipcode"] = data.groupby("zipcode")["lu_category_8"].transform('sum')
data["total_i_zipcode"] = data.groupby("zipcode")["lu_category_9"].transform('sum')
data["total_r1_zipcode"] = data.groupby("zipcode")["lu_category_10"].transform('sum')
data["total_r2_zipcode"] = data.groupby("zipcode")["lu_category_11"].transform('sum')
data["total_r3_zipcode"] = data.groupby("zipcode")["lu_category_12"].transform('sum')
data["total_r4_zipcode"] = data.groupby("zipcode")["lu_category_13"].transform('sum')
data["total_rc_zipcode"] = data.groupby("zipcode")["lu_category_14"].transform('sum')
data["total_rl_zipcode"] = data.groupby("zipcode")["lu_category_15"].transform('sum')

data["owner_occ_zipcode"] = data.groupby("zipcode")["own_occ"].transform('sum')

In [69]:
lu_cols = [col for col in data.columns if "lu_category_" in col]

for col in lu_cols:
#     data[f"{col}_total"] = data.groupby("zipcode")[col].transform('sum')
    data.drop(columns=[col], inplace=True)

In [70]:
data.drop(columns=["gis_id",
                   "st_num",
#                    "st_name",
                   "st_name_suf",
                   "lu",
#                    "lu_category",
                   "own_occ",
                   "av_land",
                   "av_bldg",
                   "av_total",
                   "gross_tax"], 
           inplace=True)
data = data.drop_duplicates(subset="zip_num_street")
data.reset_index(drop=True, inplace=True)

In [71]:
data_cols = list(data.columns)

filename = "../assets/variables/data_cols"
outfile = open(filename, "wb")
pickle.dump(data_cols, outfile)
outfile.close()

In [72]:
data

,st_name,zipcode,lu_category,num_street,zip_street,zip_num_street,land_value,bldg_value,total_value,gross_tax_amount,...,total_e_zipcode,total_ea_zipcode,total_i_zipcode,total_r1_zipcode,total_r2_zipcode,total_r3_zipcode,total_r4_zipcode,total_rc_zipcode,total_rl_zipcode,owner_occ_zipcode
0,beacon,02108,4,87 beacon,beacon 02108,87 beacon 02108,0,7842200,7842200,8265679,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
1,beacon,02108,6,88 beacon,beacon 02108,88 beacon 02108,0,12721500,12721500,13408461,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
2,beacon,02108,0,89 beacon,beacon 02108,89 beacon 02108,1373400,1588100,2961500,3121421,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
3,beacon,02108,6,90 beacon,beacon 02108,90 beacon 02108,0,7965700,7965700,8395849,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
4,beacon,02108,6,91 beacon,beacon 02108,91 beacon 02108,0,9757250,9757250,10284141,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97510,lake,02135,4,12 lake,lake 02135,12 lake 02135,0,462200,462200,487159,...,329.0,1.0,16.0,1213.0,1929.0,597.0,178.0,96.0,232.0,3751
97511,knowles,02135,7,knowles,knowles 02135,knowles 02135,223500,0,223500,0,...,329.0,1.0,16.0,1213.0,1929.0,597.0,178.0,96.0,232.0,3751
97512,lake,02135,12,4 lake,lake 02135,4 lake 02135,316200,706900,1023100,1078347,...,329.0,1.0,16.0,1213.0,1929.0,597.0,178.0,96.0,232.0,3751
97513,commonwealth,02135,2,2193 2201 commonwealth,commonwealth 02135,2193 2201 commonwealth 02135,760800,1737700,2498500,6246250,...,329.0,1.0,16.0,1213.0,1929.0,597.0,178.0,96.0,232.0,3751


In [ ]:
# ==================================

In [ ]:
# OLD- THIS APPROACH WAS WHEN ONLY ZIPCODE WAS CONSIDERED

In [ ]:
# data["zipcode"] = data["zipcode"].astype(str)
# data["zipcode"] = data["zipcode"].apply(lambda x: "0" + x[:4])  

In [ ]:
# GROUPING THE CONTINUOUS VARIABLES

# data["zip_av_land"] = data.groupby("zipcode")["av_land"].transform('sum')
# data["zip_av_bldg"] = data.groupby("zipcode")["av_bldg"].transform('sum')
# data["zip_av_total"] = data.groupby("zipcode")["av_total"].transform('sum')
# data["zip_gross_tax"] = data.groupby("zipcode")["gross_tax"].transform('sum')

# INVALUABLE, BUT THE APPROACH SHOULD BE GEOGRAPHIC SPECIFICITY
# data["zip_street"] = data[["zipcode", "st_name"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
    # data["zip_street_own_occ"] = data[["zip_street", "own_occ"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
    # data["zip_street_lu"] = data[["zip_street", "lu"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
    # data["zip_street_own_occ_lu"] = data[["zip_street_own_occ", "lu"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
# https://stackoverflow.com/questions/44497671/pandas-combine-columns-without-duplicates-find-unique-words-after-combining
# -------------------------------------- 

In [ ]:
# lu_cat_dict = {
#     0: "a",
#     1: "ah",
#     2: "c",
#     3: "cc", 
#     4: "cd", 
#     5: "cl",
#     6: "cm",
#     7: "e",
#     8: "ea",
#     9: "i",
#     10: "r1",
#     11: "r2",
#     12: "r3",
#     13: "r4",
#     14: "rc",
#     15: "rl"
# }

In [ ]:
# GROUPING THE CATEGORICAL VARIABLES BY ZIPCODE TOTALS
# data["total_a"] = data.groupby("zipcode")["lu_category_0"].transform('sum')
# data["total_ah"] = data.groupby("zipcode")["lu_category_1"].transform('sum')
# data["total_c"] = data.groupby("zipcode")["lu_category_2"].transform('sum')
# data["total_cc"] = data.groupby("zipcode")["lu_category_3"].transform('sum')
# data["total_cd"] = data.groupby("zipcode")["lu_category_4"].transform('sum')
# data["total_cl"] = data.groupby("zipcode")["lu_category_5"].transform('sum')
# data["total_cm"] = data.groupby("zipcode")["lu_category_6"].transform('sum')
# data["total_e"] = data.groupby("zipcode")["lu_category_7"].transform('sum')
# data["total_ea"] = data.groupby("zipcode")["lu_category_8"].transform('sum')
# data["total_i"] = data.groupby("zipcode")["lu_category_9"].transform('sum')
# data["total_r1"] = data.groupby("zipcode")["lu_category_10"].transform('sum')
# data["total_r2"] = data.groupby("zipcode")["lu_category_11"].transform('sum')
# data["total_r3"] = data.groupby("zipcode")["lu_category_12"].transform('sum')
# data["total_r4"] = data.groupby("zipcode")["lu_category_13"].transform('sum')
# data["total_rc"] = data.groupby("zipcode")["lu_category_14"].transform('sum')
# data["total_rl"] = data.groupby("zipcode")["lu_category_15"].transform('sum')

# data["owner_occupied"] = data.groupby("zipcode")["own_occ"].transform('sum')

In [ ]:
# lu_cols = [col for col in data.columns if "lu_category_" in col]

# for col in lu_cols:
# #     data[f"{col}_total"] = data.groupby("zipcode")[col].transform('sum')
#     data.drop(columns=[col], inplace=True)

In [87]:
# data.drop(columns=["gis_id",
#                    "st_num",
#                    "st_name",
#                    "st_name",
#                    "st_name_suf",
#                    "lu",
#                    "lu_category",
#                    "own_occ",
#                    "av_land",
#                    "av_bldg",
#                    "av_total",
#                    "gross_tax"], 
#            inplace=True)
# data = data.drop_duplicates(subset="zipcode")
# data.reset_index(drop=True, inplace=True)

In [ ]:
# data_cols = list(data.columns)

# filename = "../assets/variables/data_cols"
# outfile = open(filename, "wb")
# pickle.dump(data_cols, outfile)
# outfile.close()

### pw_df

In [ ]:
# ========================

In [73]:
pw_df["pw_num_street"] = pw_df[["st_num", "st_name"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
pw_df["pw_zip_street"] = pw_df[["st_name", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
pw_df["pw_zip_num_street"] = pw_df[["pw_num_street", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)

In [74]:
# CREATE COLUMN OF TOTAL 
pw_df["pw_viol_count_prop"] = pw_df.groupby("pw_zip_num_street")["pw_violation"].transform('sum')
pw_df["pw_viol_count_street"] = pw_df.groupby("pw_zip_street")["pw_violation"].transform('sum')
pw_df["pw_viol_count_zip"] = pw_df.groupby("zipcode")["pw_violation"].transform("sum")

# GROUPING THE CONTINUOUS VARIABLES
pw_df["pw_fine_value_prop"] = pw_df.groupby("pw_zip_num_street")["value"].transform('sum')
pw_df["pw_fine_value_street"] = pw_df.groupby("pw_zip_street")["value"].transform('sum')
pw_df["pw_fine_value_prop"] = pw_df.groupby("zipcode")["value"].transform('sum')

In [75]:
pw_df["status"].value_counts()

0    25806
1    21792
Name: status, dtype: int64

In [76]:
pw_df["pw_open_status_prop"] = pw_df.groupby("pw_zip_num_street")["status"].transform('sum')
pw_df["pw_open_status_street"] = pw_df.groupby("pw_zip_street")["status"].transform('sum')
pw_df["pw_open_status_zip"] = pw_df.groupby("zipcode")["status"].transform('sum')
pw_df.rename(columns={"value": "pw_viol_value_prop"}, inplace=True)

description_cols = [col for col in pw_df.columns if "description_category_" in col]

for col in description_cols:
    pw_df[f"{col}_total"] = pw_df.groupby("pw_zip_num_street")[col].transform('sum')
    pw_df.drop(columns=[col], inplace=True)

pw_df.drop(columns=[
    "st_num",
    "st_name",
    "status",
    "description",
    "description_category",
    "pw_violation"
#     "value"
],
          inplace=True)

pw_df.zipcode.str.replace("2120", "02120")
pw_df = pw_df[pw_df["zipcode"] != " "]
pw_df = pw_df.drop_duplicates(subset="pw_zip_num_street")
pw_df.reset_index(drop=True, inplace=True)

In [77]:
pw_cols = list(pw_df.columns)

filename = "../assets/variables/pw_cols"
outfile = open(filename, "wb")
pickle.dump(pw_cols, outfile)
outfile.close()

In [ ]:
# ========================

In [ ]:
# # CREATE COLUMN OF TOTAL ESTIMATED PROPERTY LOSS
# pw_df["pw_violation_count"] = pw_df.groupby("zipcode")["pw_violation"].transform('sum')

# # GROUPING THE CONTINUOUS VARIABLES

# pw_df["pw_fine_value"] = pw_df.groupby("zipcode")["value"].transform('sum')

In [ ]:
# pw_df["status"].value_counts()

In [ ]:
# pw_df["pw_open_status_total"] = pw_df.groupby("zipcode")["status"].transform('sum')

# description_cols = [col for col in pw_df.columns if "description_category_" in col]

# for col in description_cols:
#     pw_df[f"{col}_total"] = pw_df.groupby("zipcode")[col].transform('sum')
#     pw_df.drop(columns=[col], inplace=True)

# pw_df.drop(columns=[
#     "st_name",
#     "status",
#     "description",
#     "description_category",
#     "pw_violation",
#     "value"
# ],
#           inplace=True)

# pw_df.zipcode.str.replace("2120", "02120")
# pw_df = pw_df[pw_df["zipcode"] != " "]
# pw_df = pw_df.drop_duplicates(subset="zipcode")
# pw_df.reset_index(drop=True, inplace=True)

In [ ]:
# pw_cols = list(pw_df.columns)

# filename = "../assets/variables/pw_cols"
# outfile = open(filename, "wb")
# pickle.dump(pw_cols, outfile)
# outfile.close()

### fire_df

In [ ]:
# =======================

In [78]:
fire_df["zipcode"] = fire_df["zipcode"].astype(str)
fire_df["zipcode"] = fire_df["zipcode"].apply(lambda x: "0" + x)  

In [79]:
fire_df["fire_num_street"] = fire_df[["st_num", "st_name"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
fire_df["fire_zip_street"] = fire_df[["st_name", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
fire_df["fire_zip_num_street"] = fire_df[["fire_num_street", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)

fire_df["fire_prop_loss_prop"] = fire_df.groupby("fire_zip_num_street")["estimated property loss"].transform("sum")
fire_df["fire_prop_loss_street"] = fire_df.groupby("fire_zip_street")["estimated property loss"].transform('sum')
fire_df["fire_prop_loss_zip"] = fire_df.groupby("zipcode")["estimated property loss"].transform('sum')

# fire_df["fire_prop_loss_street"] = fire_df.groupby("fire_zip_num_street")["estimated property loss"].transform("sum")

fire_df["fire_content_loss_prop"] = fire_df.groupby("fire_zip_num_street")["estimated content loss"].transform("sum")
fire_df["fire_content_loss_street"] = fire_df.groupby("fire_zip_street")["estimated content loss"].transform('sum')
fire_df["fire_content_loss_zip"] = fire_df.groupby("zipcode")["estimated content loss"].transform('sum')

fire_df = fire_df.drop_duplicates(subset="fire_zip_num_street")
fire_df = fire_df.drop(columns=["st_name", 
                                "st_num",
                                "had_incident"])
#                                 "estimated property loss",
#                                 "estimated content loss"])
fire_df.reset_index(drop=True, inplace=True)

In [80]:
fire_cols = list(fire_df.columns)

filename = "../assets/variables/fire_cols"
outfile = open(filename, "wb")
pickle.dump(fire_cols, outfile)
outfile.close()

In [ ]:
# ============================

In [140]:
# fire_df["zipcode"] = fire_df["zipcode"].astype(str)
# fire_df["zipcode"] = fire_df["zipcode"].apply(lambda x: "0" + x)  

In [ ]:
# # CREATE COLUMN OF TOTAL ESTIMATED PROPERTY LOSS
# fire_df["fire_property_loss"] = fire_df.groupby("zipcode")["estimated property loss"].transform('sum')
# fire_df["fire_content_loss"] = fire_df.groupby("zipcode")["estimated content loss"].transform('sum')
# fire_df["total_fire_incidents"] = fire_df.groupby("zipcode")["had_incident"].transform('sum')

# fire_df = fire_df.drop_duplicates(subset="zipcode")
# fire_df = fire_df.drop(columns=["st_name", 
#                                 "had_incident",
#                                 "estimated property loss",
#                                 "estimated content loss"])
# fire_df.reset_index(drop=True, inplace=True)

In [ ]:
# fire_cols = list(fire_df.columns)

# filename = "../assets/variables/fire_cols"
# outfile = open(filename, "wb")
# pickle.dump(fire_cols, outfile)
# outfile.close()

### requests_df

In [81]:
requests_df["zipcode"] = requests_df["zipcode"].astype(str)
requests_df["zipcode"] = requests_df["zipcode"].apply(lambda x: "0" + x[:4])  

requests_df["req_num_street"] = requests_df[["st_num", "st_name"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
requests_df["req_zip_street"] = requests_df[["st_name", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)
requests_df["req_zip_num_street"] = requests_df[["req_num_street", "zipcode"]].astype(str).apply(lambda x: ' '.join(pd.unique(' '.join(x).split())),axis=1)

# CREATE COLUMN OF TOTAL REQUESTS MADE TO THE CITY
requests_df["requests_total_prop"] = requests_df.groupby("req_zip_num_street")["had_request"].transform('sum')
requests_df["requests_total_street"] = requests_df.groupby("req_zip_street")["had_request"].transform('sum')
requests_df["requests_total_zip"] = requests_df.groupby("zipcode")["had_request"].transform('sum')

requests_df["req_open_status_prop"] = requests_df.groupby("req_zip_num_street")["case_status"].transform('sum')
requests_df["req_open_status_street"] = requests_df.groupby("req_zip_street")["case_status"].transform('sum')
requests_df["req_open_status_zip"] = requests_df.groupby("zipcode")["case_status"].transform('sum')

reason_col = [col for col in requests_df.columns if "reason_category_" in col]
source_col = [col for col in requests_df.columns if "source_category_" in col]

for col in reason_col:
    requests_df[f"req_{col}_total_prop"] = requests_df.groupby("req_zip_num_street")[col].transform('sum')
for col in reason_col:
    requests_df[f"req_{col}_total_street"] = requests_df.groupby("req_zip_street")[col].transform('sum')
for col in reason_col:
    requests_df[f"req_{col}_total_zip"] = requests_df.groupby("zipcode")[col].transform('sum')
    requests_df.drop(columns=[col], inplace=True)

for col in source_col:
    requests_df[f"req_{col}_total_prop"] = requests_df.groupby("req_zip_num_street")[col].transform('sum')
for col in source_col:
    requests_df[f"req_{col}_total_street"] = requests_df.groupby("req_zip_street")[col].transform('sum')
for col in source_col:
    requests_df[f"req_{col}_total_zip"] = requests_df.groupby("zipcode")[col].transform('sum')
    requests_df.drop(columns=[col], inplace=True)
    


requests_df= requests_df.drop_duplicates(subset="req_zip_num_street")
requests_df.drop(columns=[
    "st_name",
#     "open_dt",
    "case_status",
    "reason",
    "reason_category",
    "source",
    "source_category",
    "had_request",
],
          inplace=True)
requests_df.reset_index(drop=True, inplace=True)

In [82]:
# requests_list = list(requests_df["zipcode"])
# data_list = list(data["data_zipcode"])

# for zipcode in requests_list:
#     if zipcode not in data_list:
#         print(zipcode)

In [83]:
requests_cols = list(requests_df.columns)

filename = "../assets/variables/requests_cols"
outfile = open(filename, "wb")
pickle.dump(requests_cols, outfile)
outfile.close()

## Check Files

### data

In [84]:
print(data.shape)
print(data.isna().sum().sum())
data.head()

(97515, 61)
0


,st_name,zipcode,lu_category,num_street,zip_street,zip_num_street,land_value,bldg_value,total_value,gross_tax_amount,...,total_e_zipcode,total_ea_zipcode,total_i_zipcode,total_r1_zipcode,total_r2_zipcode,total_r3_zipcode,total_r4_zipcode,total_rc_zipcode,total_rl_zipcode,owner_occ_zipcode
0,beacon,02108,4,87 beacon,beacon 02108,87 beacon 02108,0,7842200,7842200,8265679,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
1,beacon,02108,6,88 beacon,beacon 02108,88 beacon 02108,0,12721500,12721500,13408461,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
2,beacon,02108,0,89 beacon,beacon 02108,89 beacon 02108,1373400,1588100,2961500,3121421,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
3,beacon,02108,6,90 beacon,beacon 02108,90 beacon 02108,0,7965700,7965700,8395849,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651
4,beacon,02108,6,91 beacon,beacon 02108,91 beacon 02108,0,9757250,9757250,10284141,...,76.0,0.0,0.0,220.0,23.0,13.0,30.0,41.0,4.0,651


### pw violations

In [86]:
print(pw_df.shape)
print(pw_df.isna().sum().sum())
pw_df.head()

(17629, 53)
0


,zipcode,pw_viol_value_prop,pw_num_street,pw_zip_street,pw_zip_num_street,pw_viol_count_prop,pw_viol_count_street,pw_viol_count_zip,pw_fine_value_prop,pw_fine_value_street,...,description_category_30_total,description_category_31_total,description_category_32_total,description_category_33_total,description_category_34_total,description_category_35_total,description_category_36_total,description_category_37_total,description_category_38_total,description_category_39_total
0,02118,25,21 concord,concord 02118,21 concord 02118,5,182,3184,119120,6975,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,02115,25,196 saint botolph,saint botolph 02115,196 saint botolph 02115,18,134,2323,93270,4290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02108,25,68 pinckney,pinckney 02108,68 pinckney 02108,3,4,369,14865,100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,02114,250,165 friend,friend 02114,165 friend 02114,2,8,2166,65580,1025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,02113,25,263 north,north 02113,263 north 02113,7,231,2882,90375,10600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### fire incidents

In [87]:
print(fire_df.shape)
print(fire_df.isna().sum().sum())
fire_df.head()

(21051, 12)
0


,zipcode,estimated property loss,estimated content loss,fire_num_street,fire_zip_street,fire_zip_num_street,fire_prop_loss_prop,fire_prop_loss_street,fire_prop_loss_zip,fire_content_loss_prop,fire_content_loss_street,fire_content_loss_zip
0,02120,0.0,0.0,800 parker,parker 02120,800 parker 02120,0.0,0.0,663550.0,0.0,0.0,95020.0
1,02119,0.0,0.0,39 roxbury,roxbury 02119,39 roxbury 02119,0.0,16250.0,3281551.0,0.0,1750.0,852006.0
2,02119,0.0,0.0,131 walnut,walnut 02119,131 walnut 02119,2500.0,16500.0,3281551.0,2500.0,9600.0,852006.0
3,02115,0.0,0.0,90 gainsborough,gainsborough 02115,90 gainsborough 02115,10.0,10.0,112506.0,10.0,10.0,59162.0
4,02115,0.0,0.0,95 gainsborough,gainsborough 02115,95 gainsborough 02115,0.0,10.0,112506.0,0.0,10.0,59162.0


In [55]:
fire_df.isna().sum()

zipcode                     0
estimated property loss     0
estimated content loss      0
fire_num_street             0
fire_zip_street             0
fire_zip_num_street         0
fire_prop_loss_prop         0
fire_prop_loss_street       0
fire_prop_loss_zip          0
fire_content_loss_prop      0
fire_content_loss_street    0
fire_content_loss_zip       0
dtype: int64

### 311 requests

In [88]:
print(requests_df.shape)
print(requests_df.isna().sum().sum())
requests_df.head()

(68609, 149)
0


,st_num,zipcode,req_num_street,req_zip_street,req_zip_num_street,requests_total_prop,requests_total_street,requests_total_zip,req_open_status_prop,req_open_status_street,...,req_source_category_2_total_street,req_source_category_3_total_street,req_source_category_4_total_street,req_source_category_5_total_street,req_source_category_0_total_zip,req_source_category_1_total_zip,req_source_category_2_total_zip,req_source_category_3_total_zip,req_source_category_4_total_zip,req_source_category_5_total_zip
0,6-8,02124,6-8 school st,6-8 school st 02124,6-8 school st 02124,9,9,14465,0,0,...,0.0,0.0,0.0,0.0,3798.0,1073.0,8169.0,637.0,788.0,0.0
1,195,02130,195 south st,195 south st 02130,195 south st 02130,1,1,11490,0,0,...,1.0,0.0,0.0,0.0,4574.0,967.0,4829.0,411.0,705.0,4.0
2,34,02122,34 ridgewood st,34 ridgewood st 02122,34 ridgewood st 02122,6,6,7438,0,0,...,6.0,0.0,0.0,0.0,2638.0,419.0,3773.0,211.0,396.0,1.0
3,3,02124,3 school st,3 school st 02124,3 school st 02124,25,25,14465,1,1,...,13.0,0.0,0.0,0.0,3798.0,1073.0,8169.0,637.0,788.0,0.0
4,22,02127,22 ward st,22 ward st 02127,22 ward st 02127,5,5,15029,0,0,...,0.0,0.0,0.0,0.0,8407.0,900.0,4907.0,324.0,488.0,3.0


# Combine into One DataFrame

In [90]:
# RENAME COLUMNS FOR THE MERGE

data.rename(columns={"zipcode": "data_zipcode"}, inplace=True)
fire_df.rename(columns={"zipcode": "fire_zipcode"}, inplace=True)
pw_df.rename(columns={"zipcode": "pw_zipcode"}, inplace=True)
requests_df.rename(columns={"zipcode": "requests_zipcode"}, inplace=True)

#=========================================================================================
# CHANGE IT ALL BACK

# data.rename(columns={"data_zipcode": "zipcode"}, inplace=True)
# fire_df.rename(columns={"fire_zipcode": "zipcode"}, inplace=True);
# pw_df.rename(columns={"pw_zipcode": "zipcode"}, inplace=True);

In [91]:
# INSTANTIATE A NEW DATAFRAME TO MERGE INTO
main = pd.DataFrame()

# MERGE DATA WITH PW VIOLATIONS
main = pd.merge(data, pw_df, 
                   how="left", 
                   left_on=["zip_num_street"],
                   right_on=["pw_zip_num_street"])

# MERGE MAIN WITH FIRE INCIDENTS
main = pd.merge(main, fire_df,
                how="left",
                left_on=["zip_num_street"],
                right_on=["fire_zip_num_street"])

# MERGE MAIN WITH 311 REQUESTS
main = pd.merge(main, requests_df,
                how="left",
                left_on=["zip_num_street"],
                right_on=["req_zip_num_street"])

In [92]:
# main["data_zipcode"] = df[col].cat.codes
main["label_zip"] = main["data_zipcode"].astype("category")
main["label_zip"] = main["label_zip"].cat.codes + 1 # SHIFTS UP THE LABEL COLUMN UP ONE VALUE
main.rename(columns={"data_zipcode": "zipcode"}, inplace=True);

In [93]:
main["label_street"] = main["st_name"].astype("category")
main["label_street"] = main["label_street"].cat.codes + 1 # SHIFTS UP THE LABEL COLUMN UP ONE VALUE
main.rename(columns={"data_zipcode": "zipcode"}, inplace=True);

In [94]:
main.drop(columns=[
    "st_num",
    "st_name",
    "num_street",
    "pw_num_street",
    "pw_zipcode",
    "pw_zip_street",
    "pw_zip_num_street",
    "fire_zipcode",
    "fire_num_street",
    "fire_zip_street",
    "fire_zip_num_street",
    "requests_zipcode",
    "req_num_street",
    "req_zip_street",
    "req_zip_num_street"
], inplace=True)

In [95]:
# NEED TO REVISIT THIS
main.fillna(value=0, inplace=True);

# THE IDEA IS THAT, IN THEORY, IF NAN NOT IN MAIN BUT IN OTHER DF
# THEN THERE ARE NO METRICS TO HAVE BEEN REPORTED IN THAT DISTRICT/ZIP

In [96]:
print(data.shape)
print(pw_df.shape)
print(fire_df.shape)
print()
print("Number zipcodes not shared in data and pw_df:")
print(data.shape[0] - pw_df.shape[0])
print()
print("Number zipcodes not shared in data and fire_df:")
print(data.shape[0] - fire_df.shape[0])
print()
print("Number of null in main:")
print(main.isna().sum().sum())

(97515, 61)
(17629, 53)
(21051, 12)

Number zipcodes not shared in data and pw_df:
79886

Number zipcodes not shared in data and fire_df:
76464

Number of null in main:
0


In [99]:
main

,zipcode,lu_category,zip_street,zip_num_street,land_value,bldg_value,total_value,gross_tax_amount,total_a_prop,total_ah_prop,...,req_source_category_4_total_street,req_source_category_5_total_street,req_source_category_0_total_zip,req_source_category_1_total_zip,req_source_category_2_total_zip,req_source_category_3_total_zip,req_source_category_4_total_zip,req_source_category_5_total_zip,label_zip,label_street
0,02108,4,beacon 02108,87 beacon 02108,0,7842200,7842200,8265679,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,274
1,02108,6,beacon 02108,88 beacon 02108,0,12721500,12721500,13408461,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,274
2,02108,0,beacon 02108,89 beacon 02108,1373400,1588100,2961500,3121421,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,274
3,02108,6,beacon 02108,90 beacon 02108,0,7965700,7965700,8395849,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,274
4,02108,6,beacon 02108,91 beacon 02108,0,9757250,9757250,10284141,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97510,02135,4,lake 02135,12 lake 02135,0,462200,462200,487159,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,1908
97511,02135,7,knowles 02135,knowles 02135,223500,0,223500,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,1897
97512,02135,12,lake 02135,4 lake 02135,316200,706900,1023100,1078347,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,1908
97513,02135,2,commonwealth 02135,2193 2201 commonwealth 02135,760800,1737700,2498500,6246250,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,782


In [101]:
main["zipcode"].nunique()

34

# Save Files

In [102]:
# data.to_csv("../data/clean-data/data-engineered.csv")
# pw_df.to_csv("../data/clean-data/pw_df-engineered.csv")
# fire_df.to_csv("../data/clean-data/fire_df-engineered.csv")
# requests_df.to_csv("../data/clean-data/requests_df-engineered.csv")

In [ ]:
# data_1.to_csv("../data/clean-data/data_1-eng.csv")
# pd.read_csv("../data/clean-data/data_1-eng.csv")

In [98]:
# THE NEW CLEAN, EDA AND MODEL READY DATAFRAME
main.to_csv("../data/clean-data/main-engineered.csv")